## Equalizadores

Um equalizador é um dispositivo que trabalha diretamente com as frequências. Diferentemente dos filtros passa-baixa, passa-alta, passa-faixa e rejeita-faixa, que reduzem a amplitude do espectro de áudio acima ou abaixo de uma determinada frequência de corte, o equalizador molda o espectro de áudio ao realçar certas faixas de frequência enquanto deixa outras faixas inalteradas.

Normalmente, o objetivo de um equalizador é melhorar a qualidade sonora em sistemas de reprodução de som, bem como aumentar o ganho acústico no caso de sistemas de reforço sonoro. Antes do uso generalizado da tecnologia digital, era comum a utilização de equalizadores analógicos em telefones fixos utilizados para trunking. Nessa seção iremos demonstrar dois dos filtros equalizadores mais utilizados no mercado: o filtro shelving e o filtro peak.

<center><p>
<img src= "https://homestudiobrasil.com.br/wp-content/uploads/2017/05/Screen-Shot-2017-05-16-at-5.10.33-PM.png" width=400px>
</p>
<p>
    <font size=2><em>Figura 1 - Imagem de <a href="https://homestudiobrasil.com.br/2017/05/16/equalizadores-entendendo-seu-funcionamento/">homestudiobrasil</a></em></font>
</p></center>


Para melhor entendimento da aplicação do filtro shelving, segue a explicação de alguns termos:

- **gain**: ou ganho refere-se ao ajuste de amplitude aplicado a determinadas faixas de frequência. É uma medida da amplificação ou atenuação do sinal em uma faixa específica de frequências.
- **boost**: o termo "boost" refere-se ao aumento do nível de determinadas frequências específicas
- **cut**: o termo "cut" refere-se à redução do nível de determinadas frequências específicas

Importaremos todas as funções apresentadas no capítulo de introdução e o módulo **math** próprio do Python para utilizar as funções matemáticas tangente e cosseno e a constante matemática π.

In [ ]:
%run -i 1introducao.ipynb

from math import tan, pi, cos

##### Shelving 
**Low Shelf**: Este tipo de equalizador leva consigo todas as frequências abaixo da frequência selecionada. É amplamente utilizado quando se deseja obter um resultado que vá além de uma frequência ou região específica, com o objetivo de ressaltar ou atenuar todos os **graves** de forma proporcional.

<center><p>
<img src= "https://d2p7ncqs8iaoqd.cloudfront.net/2019/06/12165913/Low-Shelving1.gif" width=400px>
</p>
<p>
    <font size=2><em>Figura 2 - Imagem de <a href="https://www.avmakers.com.br/blog/equalizador-o-que-e-e-qual-a-sua-utilidade">AvMakers</a></em></font>
</p></center>

**High Shelf**: Semelhante ao Low Shelf, o High Shelf também leva consigo todas as frequências abaixo da frequência selecionada. É utilizado quando se pretende obter um resultado que vá além de uma frequência ou região específica, com o objetivo de ressaltar ou atenuar todos os **agudos** de forma proporcional.

<center><p>
<img src= "https://d2p7ncqs8iaoqd.cloudfront.net/2019/06/12165943/Hi-Shelving1.gif" width=400px>
</p>
<p>
    <font size=2><em>Figura 3 - Imagem de <a href="https://www.avmakers.com.br/blog/equalizador-o-que-e-e-qual-a-sua-utilidade">AvMakers</a></em></font>
</p></center>


Dentro do filtro shelving, temos:

- **normal_cutoff**: ou frequência de corte se refere à frequência na qual o equalizador começa a afetar o sinal de áudio, seja amplificando, atenuando ou filtrando determinadas faixas de frequência.
- **filter_class**: define se o filtro será low shelf ou high shelf.

Abaixo a função que definirá nosso filtro shelving.

In [ ]:
def shelving(input_signal, normal_cutoff, gain, filter_class):
    # inicialização do vetor de saída
    output_signal = [0] * len(input_signal)

    # paramêtros das eq. diferenciais 
    V0 = 10**(gain/20)
    H0 = V0 - 1

    # boost
    if gain >= 0:
        c = (tan(pi*normal_cutoff/2) - 1)/(tan(pi*normal_cutoff/2)+1)
    # cut
    else:
        c = (tan(pi*normal_cutoff/2) - V0)/(tan(pi*normal_cutoff/2)+V0)

    xh = 0

    # aplicação de eq. diferencias do filtro
    for n in range(len(input_signal)):
        xh_new = input_signal[n] - c*xh
        ap_y = c * xh_new + xh
        xh = xh_new
        if filter_class == 'lowpass':
            output_signal[n] = 0.5 * H0 * (input_signal[n] + ap_y) + input_signal[n]
        else:
            output_signal[n] = 0.5 * H0 * (input_signal[n] - ap_y) + input_signal[n]

    # calcula a resposta em frequência do filtro.
    w, h = freqz(output_signal, input_signal, fs=44100)

    return output_signal, w, h


##### Equalizador Paramétrico (Peak) 
Neste tipo de equalizador, é possível selecionar a frequência central a ser manipulada, a largura de banda e a amplitude dessa banda (ganho/atenuação). É uma ferramenta versátil que permite ajustes mais precisos e controlados em determinadas frequências específicas.

<center><p>
<img src= "https://d2p7ncqs8iaoqd.cloudfront.net/2019/06/12165651/Eq-Parame%CC%81trico1.gif" width=400px>
</p>
<p>
    <font size=2><em>Figura 4 - Imagem de <a href="https://www.avmakers.com.br/blog/equalizador-o-que-e-e-qual-a-sua-utilidade">AvMakers</a></em></font>
</p></center>

Dentro do filtro peak, temos:

- **center_frequency**: ou frequência central, refere a frequência em que o equalizador tem o seu maior efeito. É a frequência em torno da qual a banda de equalização é aplicada.
- **bandwidth**: ou largura de banda se refere à extensão ou abrangência das frequências que serão afetadas pelo ajuste do equalizador. Também é conhecida como qualidade do filtro ou largura de banda de equalização.

Abaixo a função que definirá nosso filtro peak.

In [ ]:
def peak(input_signal, center_frequency, bandwidth, gain):
    # inicialização do vetor de saída
    output_signal = [0] * len(input_signal) 

    # paramêtros das eq. diferenciais: ganho
    V0 = 10**(gain/20)
    H0 = V0 - 1

    # boost
    if gain >= 0:
        c = (tan(pi*bandwidth/2) - 1)/(tan(pi*bandwidth/2)+1)
    # cut
    else:
        c = (tan(pi*bandwidth/2) - V0)/(tan(pi*bandwidth/2)+V0)

    # paramêtros das eq. diferenciais: centro de frequência
    d = -cos(pi*center_frequency)
    xh = [0,0]

    # aplicação de eq. diferencias do filtro
    for n in range(len(input_signal)):
        xh_new = input_signal[n] - d*(1-c)*xh[0] + c*xh[1]
        ap_y = -c * xh_new + d*(1-c)*xh[0] + xh[1]
        xh = [xh_new, xh[0]]
        output_signal[n] = 0.5 * H0 * (input_signal[n] - ap_y) + input_signal[n]

    # calcula a resposta em frequência do filtro
    w, h = freqz(output_signal, input_signal, fs=44100)

    return output_signal, w, h

Com todas as funções dos filtros prontas, vamos aplicar o filtro:

In [ ]:
def run_filter(cutoff_frequency_int, file_name, filter_function, gain, filter_class = 'lowpass', center_frequency = 0, bandwidth = 0):
    # leitura do arquivo WAV
    sample_rate, input_signal = read_audio_file(file_name)
 
    # aplicação do teorema de nyquist e normalização das frequências de corte
    nyq = 0.5 * sample_rate
    normal_cutoff = cutoff_frequency_int / nyq
    normal_center_frequency = center_frequency / nyq
    bandwidth = bandwidth / nyq

    # aplicação do filtro shelving ou peak
    if filter_function == 'shelving':
        filtered_data, w, h = shelving(input_signal, normal_cutoff, gain, filter_class)
        # plotagem da resposta em frequência do filtro
        %matplotlib inline
        plot_filter(cutoff_frequency_int, sample_rate, w, h)
    else:
        filtered_data, w, h = peak(input_signal, normal_center_frequency, bandwidth, gain)
        # plotagem da resposta em frequência do filtro
        %matplotlib inline
        plot_filter(center_frequency, sample_rate, w, h)

    
   
    # escrita do áudio de saída em arquivo
    write_audio_file(filtered_data, sample_rate, 'output_file_equalizadores.wav')

    # plotagem dos sinais de entrada e saída no domínio do tempo e da frequência
    
    %matplotlib ipympl
    plot_signal(input_signal, sample_rate, 'Sinal de Entrada')
    plot_signal(filtered_data, sample_rate, 'Sinal de Saída')
 
    # mostra os reprodutores de áudio do sinal de entrada e de saída
    display_audio(file_name, 'output_file_equalizadores.wav')

# reseta os widgets
clear_output()
out1.clear_output()
out2.clear_output()
out3.clear_output()

# roda o filtro e apresenta o widget
run_filter(3000, 'piano.wav', 'shelving', 6, 'lowpass')
widgets_equalizers()
